In [1]:
import torch
from argparse import  Namespace

In [2]:
#torch.__version__
# !nvidia-smi
torch.cuda.is_available()

True

# build MVCL model

In [3]:
from MVCL import MultiViewModel_lit, MultiViewModel
# pip install -r requirements.txt --cache-dir D:\learn\python-lib

d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



## Configuration Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `use_inner_CL` | int | 1 | Enable **inner contrastive learning** within the same modality for feature discrimination |
| `use_inter_CL` | int | 1 | Enable **inter-modal contrastive learning** between different modalities |
| `use_cls_loss_1_2` | int | 1 | Enable classification loss for modality 1 and modality 2 tasks |
| `use_fusion` | int | 1 | Enable **feature fusion mechanism** to combine multi-modal features |
| `use_fusion1D` | int | 1 | Enable **1D fusion** strategy for processing sequential feature fusion |
| `use_fusion2D` | int | 1 | Enable **2D fusion** strategy for processing spatial feature map fusion |
| `use_mse_loss` | int | 0 | Enable Mean Squared Error loss for regression tasks |
| `only_1D` | int | 0 | **Use only 1D modality**, ignoring other dimensional features |
| `only_2D` | int | 0 | **Use only 2D modality**, ignoring other dimensional features |
| `drop_layer` | float | 0.0 | Dropout rate for regularization to prevent overfitting |
| `w_con` | float | 1.0 | Weight coefficient for contrastive learning loss in total loss |
| `w_cls` | float | 1.0 | Weight coefficient for classification loss in total loss |

### Parameter Categories

#### 🎯 **Loss Function Control**
- `use_inner_CL`, `use_inter_CL`: Control different types of contrastive learning
- `use_cls_loss_1_2`: Control classification loss
- `use_mse_loss`: Control regression loss

#### 🔄 **Feature Fusion Strategy**
- `use_fusion`: Master switch for feature fusion
- `use_fusion1D`, `use_fusion2D`: Control fusion methods for different dimensions

#### 🎛️ **Modality Selection**
- `only_1D`, `only_2D`: Control whether to use only specific dimensional modalities

#### ⚖️ **Weight Balancing**
- `w_con`, `w_cls`: Balance the importance of different loss functions
- `drop_layer`: Regularization parameter

In [4]:

### default model configuration
mvcl_cfg = Namespace(
    use_inner_CL=1,
    use_inter_CL=1,
    use_cls_loss_1_2=1,
    use_fusion=1,
    use_fusion1D=1,
    use_fusion2D=1,
    use_mse_loss=0,
    only_1D=0,
    only_2D=0,
    drop_layer=0.0,
    w_con=1.0,
    w_cls=1.0,
)

In the first building, this will download the Wav2Clip model checkpoints and the WavLM model checkpoints.

In [5]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
mvcl = MultiViewModel(cfg=mvcl_cfg)

The input of our MVCL model is a torch Tensor with shape of (batch, 1, audio_length).

Take an random tensor as example. The batch size is 2, indicating this output is from processing 2 audio samples simultaneously.

In [6]:
x = torch.randn(2, 1, 48000)
res = mvcl(x)

In [7]:
for k, v in res.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: {v.shape}")
    else:
        print(f"{k}: {v}")

raw_spec: torch.Size([2, 1, 257, 257])
raw_wav_feat: torch.Size([2, 149, 768])
feature1D: torch.Size([2, 768])
feature2D: torch.Size([2, 512])
feature: torch.Size([2, 1280])
logit1D: torch.Size([2])
logit2D: torch.Size([2])
logit: torch.Size([2])


The output is a dict:

| Feature Name | Shape | Dimension | Description |
|--------------|-------|-----------|-------------|
| `raw_spec` | `[2, 1, 257, 257]` | 4D | **Raw spectrogram** - Original frequency-time representation of audio signal with 257 frequency bins and 257 time frames |
| `raw_wav_feat` | `[2, 149, 768]` | 3D | **Raw waveform features** - Sequential audio features extracted from backbone (e.g., WavLM), 149 time steps with 768-dimensional embeddings |
| `feature1D` | `[2, 768]` | 2D | **1D modality features** - the final classification feat of the 1D branch |
| `feature2D` | `[2, 512]` | 2D | **2D modality features** - the final classification feat of the 2D branch |
| `feature` | `[2, 1280]` | 2D | **Fused features** - Combined multi-modal features (1D + 2D), concatenated to 1280 dimensions (768 + 512) |
| `logit1D` | `[2]` | 1D | **1D modality logits** - Classification scores from 1D feature branch for binary classification |
| `logit2D` | `[2]` | 1D | **2D modality logits** - Classification scores from 2D feature branch for binary classification |
| `logit` | `[2]` | 1D | **Final logits** - Combined classification scores from fused features for final prediction |


**Feature Processing Pipeline**

```
Audio Input (batch, 1, 48000)
    ↓
┌─────────────────────────────────────────────────────────────┐
│                    Stage 1 (No Grad)                       │
├──────────────────────────┬──────────────────────────────────┤
│      1D Branch           │           2D Branch              │
│                          │                                  │
│ feature_model1D          │ feature_model2D                  │
│ .compute_stage1(x)       │ .compute_stage1(x, spec_aug)     │
│      ↓                   │      ↓                           │
│    wav1                  │   spec1, raw_spec               │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│                 Cross-Modal Fusion                          │
├──────────────────────────┬──────────────────────────────────┤
│   squeeze_modules[0]     │     expand_modules[0]            │
│   (wav1, spec1)          │     (wav1, spec1)                │
│      ↓                   │      ↓                           │
│   fused_wav1             │   fused_spec1                    │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│                    Stage 2                                  │
├──────────────────────────┬──────────────────────────────────┤
│ feature_model1D          │ feature_model2D                  │
│ .compute_stage2          │ .compute_stage2                  │
│ (fused_wav1)             │ (fused_spec1)                    │
│      ↓                   │      ↓                           │
│ wav2, position_bias      │   spec2                          │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│              Cross-Modal Fusion + Stage 3                   │
├──────────────────────────┬──────────────────────────────────┤
│   squeeze_modules[1]     │     expand_modules[1]            │
│   (wav2, spec2)          │     (wav2, spec2)                │
│      ↓                   │      ↓                           │
│ feature_model1D          │ feature_model2D                  │
│ .compute_stage3          │ .compute_stage3                  │
│      ↓                   │      ↓                           │
│ wav3, position_bias      │   spec3                          │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│              Cross-Modal Fusion + Stage 4                   │
├──────────────────────────┬──────────────────────────────────┤
│   squeeze_modules[2]     │     expand_modules[2]            │
│   (wav3, spec3)          │     (wav3, spec3)                │
│      ↓                   │      ↓                           │
│ feature_model1D          │ feature_model2D                  │
│ .compute_stage4          │ .compute_stage4                  │
│      ↓                   │      ↓                           │
│ wav4, position_bias      │   spec4                          │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│           Final Cross-Modal Fusion + Latent Features        │
├──────────────────────────┬──────────────────────────────────┤
│   squeeze_modules[3]     │     expand_modules[3]            │
│   (wav4, spec4)          │     (wav4, spec4)                │
│      ↓                   │      ↓                           │
│ feature_model1D          │ feature_model2D                  │
│ .compute_latent_feature  │ .compute_latent_feature          │
│      ↓                   │      ↓                           │
│ wav5, raw_wav_feat       │   spec5                          │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│                Feature Normalization                        │
├──────────────────────────┬──────────────────────────────────┤
│   norm_feat(wav5)        │   norm_feat(spec5)               │
│      ↓                   │      ↓                           │
│   feature1D [B, 768]     │   feature2D [B, 512]             │
│      ↓                   │      ↓                           │
│   cls1D(feature1D)       │   cls2D(feature2D)               │
│      ↓                   │      ↓                           │
│   logit1D [B]            │   logit2D [B]                    │
└──────────────────────────┴──────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────────┐
│                   Multi-Modal Fusion                        │
│                                                             │
│        concat([wav5, spec5], dim=-1)                        │
│                     ↓                                       │
│              norm_feat(concat)                              │
│                     ↓                                       │
│               feature [B, 1280]                             │
│                     ↓                                       │
│              cls_final(feature)                             │
│                     ↓                                       │
│                logit [B]                                    │
└─────────────────────────────────────────────────────────────┘
```

# Lit model

> PyTorch Lightning is the deep learning framework for professional AI researchers and machine learning engineers who need maximal flexibility without sacrificing performance at scale. Lightning evolves with you as your projects go from idea to paper/production

We use [pytorch_lightning](https://lightning.ai/docs/pytorch/stable/) to train, validate, and test our model. Besides, it can also easily control the logging, model saving and callbacks.

In [8]:
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.loggers import  CSVLogger

We use the pytorch Lightning module to train the model, where we define the train step, validation/predict step, loss function and optimizer.

In [9]:
mvcl_lit = MultiViewModel_lit(cfg=mvcl_cfg)

BCE loss with label smoothing:  0.1


## Test forwarding 

In the lit model, we use the `_shared_pred` method to predict the logits of the input batch. If the stage is train, we also the the audio_transform to augment the spectrogram.

Generate a random batch:

In [10]:
batch = {
    "label": torch.randint(0, 2, (3,)),
    "audio": torch.randn(3, 1, 48000),
    "sample_rate": [16000, 16000, 16000],
}

Note, **your batch must be a dict with above keys**.

As can be seen, the `_shared_pred` output is also a dict. We use it to compute the loss
function, AUC, and ERR scores.

In [11]:
batch_res = mvcl_lit._shared_pred(batch=batch, batch_idx=0)
for key, value in batch_res.items():
    print(key, value.shape)

raw_spec torch.Size([3, 1, 257, 257])
raw_wav_feat torch.Size([3, 149, 768])
feature1D torch.Size([3, 768])
feature2D torch.Size([3, 512])
feature torch.Size([3, 1280])
logit1D torch.Size([3])
logit2D torch.Size([3])
logit torch.Size([3])
pred torch.Size([3])


# Demo training

We first build a simple dataloaders for training, where all the samples are randomly generated.

In [12]:
from callbacks import EER_Callback, BinaryAUC_Callback, BinaryACC_Callback

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

Generate a dataloader with random values for demo training.

In [14]:
import os
import glob

# === 1. 配置实际路径 ===
# download/archive/LA
DATA_ROOT = r"D:\learn\MVCL-ADD\download\archive\LA"

# 定义输出列表文件的位置
LIST_SAVE_DIR = "data_lists"
if not os.path.exists(LIST_SAVE_DIR):
    os.makedirs(LIST_SAVE_DIR)

def generate_eval_list():
    # 协议文件路径 (CM protocols)
    protocol_path = os.path.join(
        DATA_ROOT, 
        "ASVspoof2019_LA_cm_protocols", 
        "ASVspoof2019.LA.cm.eval.trl.txt"
    )
    
    # 音频文件夹路径
    audio_dir = os.path.join(DATA_ROOT, "ASVspoof2019_LA_eval", "flac")
    
    output_path = os.path.join(LIST_SAVE_DIR, "eval_list.txt")
    
    if not os.path.exists(protocol_path):
        print(f"❌ 错误：找不到协议文件: {protocol_path}")
        return None
        
    print(f"正在处理测试集协议: {protocol_path}")
    
    count = 0
    with open(protocol_path, 'r') as f_in, open(output_path, 'w') as f_out:
        for line in f_in:
            parts = line.strip().split(' ')
            # 格式: LA_0069 LA_E_1000147 - - spoof
            audio_name = parts[1]
            label_str = parts[4]
            
            # 转换标签: bonafide(真)=1, spoof(假)=0
            label = 1 if label_str == 'bonafide' else 0
            
            # 拼接完整绝对路径
            full_path = os.path.join(audio_dir, audio_name + '.flac')
            
            # 写入一行: 路径 标签
            f_out.write(f"{full_path} {label}\n")
            count += 1
            
    print(f"✅ 生成完毕！共 {count} 条数据。")
    print(f"📄 列表文件保存在: {output_path}")
    return output_path

# 执行生成
eval_list_path = generate_eval_list()

正在处理测试集协议: D:\learn\MVCL-ADD\download\archive\LA\ASVspoof2019_LA_cm_protocols\ASVspoof2019.LA.cm.eval.trl.txt
✅ 生成完毕！共 71237 条数据。
📄 列表文件保存在: data_lists\eval_list.txt


In [28]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

# === 强制检查 soundfile 是否安装 ===
try:
    import soundfile
    print("✅ soundfile 库已检测到 (我们将强制使用它)")
except ImportError:
    # 如果这里报错，请在一个新 Cell 里运行: !pip install soundfile
    raise ImportError("❌ 严重错误: 请先安装soundfile")


# ===  定义 Dataset ===
class ASVspoofDataset(Dataset):
    def __init__(self, list_path, max_len=64000):
        """
        list_path: 第一步生成的 txt 文件路径
        max_len: 音频长度限制 (64000点 ≈ 4秒，WavLM通常需要定长输入)
        """
        self.data_list = []
        self.max_len = max_len
        
        # 读取列表
        with open(list_path, 'r') as f:
            for line in f:
                path, label = line.strip().split(' ')
                self.data_list.append((path, int(label)))
        
        print(f"📦 Dataset 加载完成，共 {len(self.data_list)} 个样本")

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        audio_path, label = self.data_list[idx]
        
        # 1. 加载音频 (torchaudio会自动处理flac)
        # waveform shape: [Channels, Time]， 指定backend后端引擎来读取音频文件
        waveform, sample_rate = torchaudio.load(audio_path, backend="soundfile")
        
        # 2. 统一长度处理 (Pad or Crop)
        curr_len = waveform.shape[1]
        if curr_len < self.max_len:
            # 如果短了，在右边补零
            pad_width = self.max_len - curr_len
            waveform = torch.nn.functional.pad(waveform, (0, pad_width))
        else:
            # 如果长了，截取前 max_len
            waveform = waveform[:, :self.max_len]
            
        # 3. 返回字典 (符合 mvcl_lit 的输入要求)
        return {
            "audio": waveform,       # [1, 64000]
            "label": label,          # 0 或 1
            "sample_rate": sample_rate
        }
    
    
# 1. 定义列表文件路径 (确保这个文件存在！)
eval_list_path = "data_lists/eval_list.txt" 

# 2. 创建真实数据的 Dataset 和 DataLoader
# 注意：这里我们只创建 test_dataloader，因为你目前只生成了 eval 的列表
test_dataset = ASVspoofDataset(list_path=eval_list_path, max_len=64000)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=16,      # 根据显存调整，推荐 8 或 16
    shuffle=False,      # 测试集不需要打乱
    num_workers=0       # Windows 下必须设为 0
)

print(f"🚀 DataLoader 准备就绪！")
print(f"   测试集样本数: {len(test_dataset)}")

✅ soundfile 库已检测到 (我们将强制使用它)
📦 Dataset 加载完成，共 71237 个样本
🚀 DataLoader 准备就绪！
   测试集样本数: 71237


In [29]:
# === 最终测试执行脚本 ===

# 1. 安全检查：如果模型或 Trainer 丢失了，自动重新初始化
if 'trainer' not in locals() or 'mvcl_lit' not in locals():
    print("⚠️ 检测到环境被重置，正在重新初始化模型和 Trainer...")
    from pytorch_lightning import Trainer
    from pytorch_lightning.loggers import CSVLogger
    from MVCL import MultiViewModel_lit
    from argparse import Namespace
    
    # 配置
    mvcl_cfg = Namespace(
        use_inner_CL=1, use_inter_CL=1, use_cls_loss_1_2=1,
        use_fusion=1, use_fusion1D=1, use_fusion2D=1,
        use_mse_loss=0, only_1D=0, only_2D=0,
        drop_layer=0.0, w_con=1.0, w_cls=1.0,
    )
    # 模型
    mvcl_lit = MultiViewModel_lit(cfg=mvcl_cfg)
    # Trainer
    trainer = Trainer(
        accelerator="gpu", 
        devices=[0], 
        enable_checkpointing=False, 
        logger=CSVLogger(save_dir="./logs", version=99) # 临时日志
    )
    print("✅ 初始化完成！")

# 2. 运行测试
print("-" * 30)
print(f"🚀 开始对 {len(test_dataloader.dataset)} 条真实数据进行推理...")
print("    (由于数据量较大，这可能需要几分钟，请耐心等待进度条走完)")
print("-" * 30)

# 正式测试
trainer.test(mvcl_lit, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


------------------------------
🚀 开始对 71237 条真实数据进行推理...
    (由于数据量较大，这可能需要几分钟，请耐心等待进度条走完)
------------------------------
Testing: |          | 0/? [00:27<?, ?it/s]


ImportError: TorchCodec is required for load_with_torchcodec. Please install torchcodec to use this function.

In [30]:
class SimpleTestDataset(Dataset):
    def __init__(self, num_samples=10):
        # Generate synthetic data similar to your example
        self.samples = []
        for _ in range(num_samples):
            self.samples.append({
                "audio": torch.randn(1, 48000),
                "label": torch.randint(0, 2, (1,)).item(),
                "sample_rate": 16000,
            })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]



# Create the dataset and dataloader
train_dataloader = DataLoader(
    SimpleTestDataset(num_samples=100),
    batch_size=3,
    shuffle=True,
)
val_dataloader = DataLoader(
    SimpleTestDataset(num_samples=50),
    batch_size=3,
    shuffle=False,
)
test_dataloader = DataLoader(
    SimpleTestDataset(num_samples=20),
    batch_size=3,
    shuffle=False,
)

We build a simple trainer to train and test our model, which uses:
- 

In [ ]:
trainer = Trainer(
    logger=CSVLogger(save_dir="./logs", version=0),
    max_epochs=4,
    callbacks=[
        BinaryACC_Callback(batch_key="label", output_key="logit"),
        BinaryAUC_Callback(batch_key="label", output_key="logit"),
        EER_Callback(batch_key="label", output_key="logit"),
    ],
    devices=[0], # use cuda:0 device
    accelerator="gpu", # use GPU acceleration
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Start training

In [ ]:
trainer.fit(mvcl_lit, train_dataloader, val_dataloaders=val_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\lightning_fabric\loggers\csv_logs.py:268: Experiment logs directory ./logs\lightning_logs\version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:658: Checkpoint directory ./logs\lightning_logs\version_0\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                            | Params | Mode 
----------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\utilities\_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\utilities\_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (34) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 3: 100%|██████████| 34/34 [00:03<00:00,  8.69it/s, v_num=0, val-clip_loss=2.210, val-mse_loss=1.000, val-cls_loss1D=0.695, val-cls_loss2D=0.695, val-cls_loss=0.695, val-contrast_loss=0.310, val-contrast_loss1D=0.330, val-contrast_loss2D=0.289, val-loss=4.910, val-acc=0.540, val-auc=0.523, val-eer=0.522, train-clip_loss=2.240, train-mse_loss=1.000, train-cls_loss1D=0.694, train-cls_loss2D=0.692, train-cls_loss=0.691, train-contrast_loss=0.294, train-contrast_loss1D=0.324, train-contrast_loss2D=0.290, train-loss=4.930, train-acc=0.560, train-auc=0.591, train-eer=0.400]

`Trainer.fit` stopped: `max_epochs=4` reached.


Epoch 3: 100%|██████████| 34/34 [00:08<00:00,  3.94it/s, v_num=0, val-clip_loss=2.210, val-mse_loss=1.000, val-cls_loss1D=0.695, val-cls_loss2D=0.695, val-cls_loss=0.695, val-contrast_loss=0.310, val-contrast_loss1D=0.330, val-contrast_loss2D=0.289, val-loss=4.910, val-acc=0.540, val-auc=0.523, val-eer=0.522, train-clip_loss=2.240, train-mse_loss=1.000, train-cls_loss1D=0.694, train-cls_loss2D=0.692, train-cls_loss=0.691, train-contrast_loss=0.294, train-contrast_loss1D=0.324, train-contrast_loss2D=0.290, train-loss=4.930, train-acc=0.560, train-auc=0.591, train-eer=0.400]


After training, you can view the logging loss in the logger file, for example `logs/lightning_logs/version_0/metrics.csv`.
![](imgs/loss.png)

## Demo Testing

After testing, the results will also saved in logger file.

In [ ]:
trainer.test(mvcl_lit, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\utilities\_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
d:\ProgramFiles\anaconda3\envs\my_project\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:00<00:00, 30.22it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test-acc          │     0.550000011920929     │
│         test-auc          │    0.43434345722198486    │
│      test-clip_loss       │    2.1738295555114746     │
│       test-cls_loss       │    0.6950899362564087     │
│      test-cls_loss1D      │    0.6951161026954651     │
│      test-cls_loss2D      │    0.6945802569389343     │
│    test-contrast_loss     │    0.3196900486946106     │
│   test-contrast_loss1D    │    0.35999998450279236    │
│   test-contrast_loss2D    │    0.27993908524513245    │
│         test-eer          │    0.6363636255264282     │
│         test-loss         │     4.898554801940918     │
│       test-mse_loss       │    1.0040218830108643     │
└───────────────────────────┴───────────────────────────┘

[{'test-clip_loss': 2.1738295555114746,
  'test-mse_loss': 1.0040218830108643,
  'test-cls_loss1D': 0.6951161026954651,
  'test-cls_loss2D': 0.6945802569389343,
  'test-cls_loss': 0.6950899362564087,
  'test-contrast_loss': 0.3196900486946106,
  'test-contrast_loss1D': 0.35999998450279236,
  'test-contrast_loss2D': 0.27993908524513245,
  'test-loss': 4.898554801940918,
  'test-acc': 0.550000011920929,
  'test-auc': 0.43434345722198486,
  'test-eer': 0.6363636255264282}]

<div class="alert alert-success">
Note, train, val, and test process will logging in the same file: `metrics.csv`.
</div>